<a href="https://colab.research.google.com/github/mariahcrestani/imersaoia_alura_25/blob/main/Tinder_de_Concursos_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
%pip -q install google-genai

In [54]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [55]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [56]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [57]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [90]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [91]:
##########################################
# --- Agente 1: Perfil do Concurseiro --- #
##########################################
def agente_perfil(resposta_usuario):

    perfil = Agent(
        name="agente_perfil",
        model="gemini-2.0-flash",
        instruction="""

Você é um agente especializado em acolher usuários e coletar informações para personalização de busca por concursos públicos. Inicie a conversa de forma amigável e faça as seguintes perguntas, uma de cada vez, garantindo que o usuário compreenda e responda a cada uma antes de prosseguir para a próxima. O objetivo é construir um perfil detalhado.
Após coletar as três respostas, compile-as em um **perfil de usuário estruturado**, utilizando os seguintes campos:
- **Escolaridade:** [Resposta do Usuário]
- **Área(s) de Interesse:** [Resposta(s) do Usuário]
- **Localidade(s) de Interesse:** [Resposta(s) do Usuário]
""",
        description="Agente que coleta informações para criar o perfil do concurseiro",
        tools=[google_search]
    )

    entrada_do_agente_perfil = f"Respostas do usuário: {resposta_usuario}"

    perfil_resultados_usuario = call_agent(perfil, entrada_do_agente_perfil)
    return perfil_resultados_usuario

In [123]:
################################################
# --- Agente 2: Buscador de Concursos --- #
################################################
def agente_buscador(perfil_resultado_usuario, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.5-flash-preview-04-17",
        # Inserir as instruções do Agente Buscador de Concursos #################################################
        instruction="""
Você é um agente especialista em pesquisa de concursos públicos. Com base no **perfil de usuário fornecido**, sua missão é realizar uma busca abrangente no Google para identificar os **10 concursos públicos mais relevantes e atualizados** que se encaixam nos critérios.

**Perfil do Usuário:**
[Perfil gerado pelo Agente 1, ex:
- Escolaridade: Superior Completo
- Área(s) de Interesse: Educação, Tecnologia
- Localidade(s) de Interesse: São Paulo (SP), Nacional
]

**Sua pesquisa deve considerar os seguintes pontos:**
1. Utilize **termos de busca variados e estratégicos** no Google, combinando escolaridade, área de interesse, localidade e palavras-chave como "concursos abertos", "editais concursos", "vagas concursos".
2. Priorize resultados de **fontes confiáveis** (sites de órgãos governamentais, bancas organizadoras, portais de notícias de concursos renomados).
3. Foque em concursos com **inscrições abertas ou iminentes** (previstas para breve).
4. Extraia as **seguintes informações** para cada concurso relevante encontrado:
    * **Nome do Órgão/Banca:** (Ex: Prefeitura de São Paulo, IBFC, CESPE)
    * **Cargo(s) Abrangido(s):** (Ex: Professor de Matemática, Analista de Sistemas, Técnico Administrativo)
    * **Escolaridade Exigida:** (Ex: Nível Superior, Nível Médio)
    * **Localidade do Concurso:** (Ex: São Paulo, Brasília)
    * **Número de Vagas (se disponível):** (Ex: 10 vagas, Cadastro de Reserva)
    * **Estimativa Salarial (se disponível):** (Ex: R$ 5.000,00 - R$ 8.000,00 ou conforme edital)
    * **Data Limite para Inscrição (se disponível):**
    * **Link para o Edital/Página Oficial:**
5. Apresente os 10 concursos encontrados em uma **lista numerada e organizada**, com as informações claras para cada item. Se não houver 10 resultados altamente relevantes, liste todos os que encontrar, explicando se o número é menor.
""",
        description="Agente Responsável pela Busca dos Concursos Relevantes",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Perfil do Usuário:{perfil_resultado_usuario}\nData de hoje: {data_de_hoje}"

    lista_de_concursos = call_agent(buscador, entrada_do_agente_buscador)
    return lista_de_concursos # Continue retornando o resultado bruto


In [124]:
######################################
# --- Agente 3: Match de Concursos --- #
######################################
def agente_matcher(perfil_resultado_usuario, lista_de_concursos):
    matcher = Agent(
        name="agente_matcher",
        model="gemini-2.5-flash-preview-04-17-thinking",
        instruction="""
        Você é um agente especialista em análise e recomendação de concursos públicos. Receberá uma **lista de concursos públicos** (gerada pelo Agente 2) e o **perfil detalhado do usuário** (gerado pelo Agente 1). Sua tarefa é analisar cuidadosamente cada concurso da lista e identificar o **concurso mais ideal ou os 2-3 mais ideais** para o usuário, justificando sua escolha.

**Perfil do Usuário:**
[Perfil gerado pelo Agente 1]

**Lista de Concursos Encontrados:**
[Lista gerada pelo Agente 2]

**Critérios de Análise para o Match:**
1.  **Escolaridade:** O concurso exige a escolaridade que o usuário possui ou é compatível?
2.  **Área de Interesse:** O cargo/área do concurso se alinha com a(s) área(s) de interesse do usuário? Priorize a correspondência direta.
3.  **Localidade:** A localidade do concurso é uma das preferências do usuário (estado, cidade, ou nacional)?
4.  **Estimativa Salarial:** O salário oferecido é atrativo e condizente com as expectativas do usuário (se inferível ou se o usuário tiver expressado preferência em alguma etapa anterior)?
5.  **Nível de Dificuldade:** O nível de dificuldade estimado do concurso (se disponível/inferível) se alinha com o que o usuário provavelmente está preparado para enfrentar, ou se a dificuldade é aceitável para o potencial retorno?
6.  **Relevância Adicional:** Considere fatores como prazo de inscrição (mais curto pode indicar maior urgência), número de vagas (mais vagas podem ser mais atrativas), e a reputação do órgão/banca.

**Sua saída deve ser:**
* Uma **breve introdução** confirmando que a análise foi realizada.
* O **concurso (ou concursos) mais ideal(is)** para o usuário, apresentando todas as informações que foram extraídas pelo Agente 2 para este(s) concurso(s).
* Uma **justificativa detalhada** explicando por que este(s) concurso(s) foi(ram) considerado(s) o(s) mais adequado(s), mencionando explicitamente como ele(s) se alinha(m) com a escolaridade, área(s) de interesse, localidade(s), **estimativa salarial** e **nível de dificuldade** do usuário.
* Se houver um empate técnico entre 2-3 concursos, apresente-os e explique as vantagens de cada um.
"""        ,
        description="Agente Especialista na Seleção do Concurso Ideal"
    )
    entrada_do_agente_matcher = f"Perfil do Usuário: {perfil_resultado_usuario}\nLista de Concursos: {lista_de_concursos}"
    # Executa o agente
    concursos_recomendados = call_agent(matcher, entrada_do_agente_matcher)
    return concursos_recomendados

In [125]:
##########################################
# --- Agente 4: Apresentado de Resultados --- #
##########################################
def agente_apresentador(concursos_recomendados):
    apresentador = Agent(
        name="agente_apresentador",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
Você é um agente especialista em comunicação e apresentação de resultados. Receberá a **recomendação de concurso(s) ideal(is)** e a **justificativa** (geradas pelo Agente 3). Sua tarefa é apresentar essa informação ao usuário de forma clara, amigável e encorajadora.

**Informações Recebidas:**
[Recomendação(ões) de concurso(s) do Agente 3, incluindo detalhes e justificativa]

**Sua saída deve ser:**
1. Uma **saudação inicial** calorosa e personalizada, direcionada ao usuário.
2. Apresente o **concurso (ou concursos) que melhor se encaixa(m)** no perfil do usuário. Para cada concurso, liste as informações essenciais de forma organizada (Órgão/Banca, Cargo, Escolaridade, Localidade, Vagas, Estimativa Salarial, Nível de Dificuldade, Prazo de Inscrição, Link).
3. **Explique de forma concisa e acessível** por que este(s) concurso(s) foi(ram) considerado(s) o(s) mais adequado(s), utilizando a justificativa fornecida pelo Agente 3, mas refraseando-a para ser mais direta e compreensível para o usuário leigo.
4. Encoraje o usuário a **clicar no link do edital** para obter mais informações e realizar a inscrição.
5. Ofereça-se para **ajudar com novas buscas** ou tirar dúvidas, caso o usuário deseje.
6. Mantenha um **tom otimista e prestativo** durante toda a comunicação.
""",
        description="Agente que formata e apresenta os resultados ao usuário"
    )
    entrada_do_agente_apresentador = f"Concursos Recomendados: {concursos_recomendados}"
    # Executa o agente
    resultado_formatado = call_agent(apresentador, entrada_do_agente_apresentador)
    return resultado_formatado

In [131]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🔥 Bem-vindo(a) ao Tinder para Concursos! 🔥")
print("Preparado(a) para dar o MATCH no concurso perfeito para você?")
print("\nPrimeiro, vamos te conhecer melhor com 3 perguntinhas rápidas:")


# --- Obter o Tópico do Usuário ---
# Pergunta 1: Escolaridade
escolaridade_usuario = input("1. Qual é a escolaridade atual (nível fundamental, médio, técnico, superior completo, pós-graduação, etc.)? ")

# Pergunta 2: Área de Interesse
area_interesse_usuario = input("2. Em qual area de interesse você busca concursos (ex: educação, saúde, direito, tecnologia, segurança pública, administrativa, fiscal, etc.)? Se tiver mais de uma, pode listar. ")

# Pergunta 3: Localidade
localidade_interesse_usuario = input("3. Qual a localidade em que você tem interesse em fazer o concurso (Estado, cidade ou \"nacional\" para todo o Brasil)? ")

# Combinar as respostas em um formato adequado para o agente
# Você pode formatar como uma string simples ou um formato mais estruturado
resposta_usuario = f"""
Escolaridade: {escolaridade_usuario}
Área(s) de Interesse: {area_interesse_usuario}
Localidade(s) de Interesse: {localidade_interesse_usuario}
"""


# Inserir lógica do sistema de agentes ################################################
if not resposta_usuario:
    print("Por favor, responda o quiz para proseguir")
else:
    print(f"Maravilha! Vamos encontrar as melhores opções para você")

    perfil_resultado_usuario = agente_perfil(resposta_usuario)
    print("\n--- 📝 Resultado do Agente 1 (Perfil do Usuário) ---\n")
    display(to_markdown(perfil_resultado_usuario))
    print("--------------------------------------------------------------")

    lista_de_concursos = agente_buscador(perfil_resultado_usuario, data_de_hoje)

    matcher_resultado = agente_matcher(perfil_resultado_usuario, lista_de_concursos)

    resultados_formatados = agente_apresentador (matcher_resultado)
    print("\n--- 📝 Resultado do Agente 4 (Resultados) ---\n")
    display(to_markdown(resultados_formatados))
    print("--------------------------------------------------------------")

🔥 Bem-vindo(a) ao Tinder para Concursos! 🔥
Preparado(a) para dar o MATCH no concurso perfeito para você?

Primeiro, vamos te conhecer melhor com 3 perguntinhas rápidas:
1. Qual é a escolaridade atual (nível fundamental, médio, técnico, superior completo, pós-graduação, etc.)? médio
2. Em qual area de interesse você busca concursos (ex: educação, saúde, direito, tecnologia, segurança pública, administrativa, fiscal, etc.)? Se tiver mais de uma, pode listar. adm
3. Qual a localidade em que você tem interesse em fazer o concurso (Estado, cidade ou "nacional" para todo o Brasil)? rj
Maravilha! Vamos encontrar as melhores opções para você

--- 📝 Resultado do Agente 1 (Perfil do Usuário) ---



> Olá! Tudo bem? Que bom ter você por aqui! Para te ajudar a encontrar os melhores concursos públicos, preparei um perfil com as suas preferências. Veja como ficou:
> 
> **Perfil do Usuário:**
> 
> *   **Escolaridade:** Médio
> *   **Área(s) de Interesse:** Administrativa (adm)
> *   **Localidade(s) de Interesse:** Rio de Janeiro (rj)
> 
> Com este perfil, posso te ajudar a encontrar os concursos mais adequados para você. Se precisar de alguma alteração ou tiver mais alguma informação para adicionar, é só me avisar! 😉
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Resultados) ---



> Olá! Tenho excelentes notícias para você! 🎉 Sua busca por concursos teve resultados incríveis, e estou aqui para te apresentar as melhores opções que encontramos que se encaixam perfeitamente no seu perfil!
> 
> Analisei com carinho as recomendações baseadas nas suas preferências (escolaridade, área de interesse e localidade), e identifiquei os concursos que mais se destacam para você neste momento. Preparei um resumo claro para facilitar sua decisão:
> 
> **As Melhores Oportunidades de Concurso Público Para Você:**
> 
> 1.  **Polícia Federal (Agente Administrativo)**
>     *   **Órgão/Banca:** Polícia Federal (Cebraspe)
>     *   **Cargo(s):** Agente Administrativo
>     *   **Escolaridade:** Nível Médio
>     *   **Localidade:** Nacional (com provas aplicadas em capitais, **incluindo o Rio de Janeiro!**)
>     *   **Vagas:** 100 para Nível Médio
>     *   **Salário Estimado:** R$ 7.444,80
>     *   **Nível de Dificuldade:** Alto
>     *   **Prazo de Inscrição:** Até 21 de maio de 2025
>     *   **Onde encontrar o edital:** Consultar site da banca Cebraspe
> 
> 2.  **UENF - Universidade Estadual do Norte Fluminense (Técnico Profissional de Nível Médio)**
>     *   **Órgão/Banca:** UENF (Instituto Selecon)
>     *   **Cargo(s):** Técnico Profissional de Nível Médio (incluindo Assistente Administrativo)
>     *   **Escolaridade:** Nível Médio
>     *   **Localidade:** **Rio de Janeiro** (UENF)
>     *   **Vagas:** 156 vagas no total (incluindo administrativo) + CR
>     *   **Salário Estimado:** R$ 2.571,58
>     *   **Nível de Dificuldade:** Moderado
>     *   **Prazo de Inscrição:** Verificar edital oficial (Edital n.º 001/2025 divulgado)
>     *   **Onde encontrar o edital:** Acessar publicações no site do Instituto Selecon
> 
> 3.  **ION Niterói - Empresa de Infraestrutura e Obras (Assistente Administrativo)**
>     *   **Órgão/Banca:** ION (Instituto Selecon)
>     *   **Cargo(s):** Diversos, incluindo **Assistente Administrativo**
>     *   **Escolaridade:** Nível Médio, Técnico e Superior (cargo de interesse é Nível Médio)
>     *   **Localidade:** **Niterói (RJ)**
>     *   **Vagas:** 58 vagas (total)
>     *   **Salário Estimado:** A partir de R$ 2.300,40 (para nível médio administrativo)
>     *   **Nível de Dificuldade:** Moderado
>     *   **Prazo de Inscrição:** **Inscrições abertas de 1º de abril a 12 de maio de 2025**. Provas em 29/06/2025.
>     *   **Onde encontrar o edital:** Acessar publicações no site do Instituto Selecon
> 
> **Por que esses concursos são ideais para você?**
> 
> Escolhi esses três por se destacarem e se alinharem aos seus objetivos:
> 
> *   A **Polícia Federal** é uma grande oportunidade para quem busca um **salário alto** e **prestígio**. O cargo de Agente Administrativo se encaixa na sua área e exige Nível Médio. O ponto positivo é que, mesmo sendo um concurso nacional, as **provas são realizadas no Rio de Janeiro**, o que facilita sua participação. Lembre-se que a lotação pode ser em outro estado, mas o esforço vale a pena pelo retorno!
> *   O concurso da **UENF** é um 'match' quase perfeito para sua busca por um cargo no **Rio de Janeiro** na **área Administrativa**, exigindo apenas **Nível Médio**. O **edital já está publicado**, o que traz segurança e clareza sobre todas as etapas. É uma oportunidade concreta e local, com um nível de dificuldade mais acessível.
> *   O concurso da **ION Niterói** também é super relevante! Ele está na **região metropolitana do Rio (Niterói)**, tem vagas para **Assistente Administrativo** de Nível Médio e o grande diferencial é que as **inscrições estão abertas AGORA** e a **prova já tem data marcada** para daqui a pouco. É uma chance mais imediata para você começar sua jornada nos concursos!
> 
> Em resumo, você tem opções excelentes: uma com **alto salário e prestígio (PF)**, uma **local com edital aberto (UENF)**, e outra **imediata com inscrições abertas (ION Niterói)**, todas na sua área de interesse e escolaridade.
> 
> Meu conselho é você dar uma olhada nos sites indicados para **acessar os editais completos**. Lá você encontra todos os detalhes de conteúdo, requisitos específicos e o passo a passo para se inscrever. Não perca os prazos, especialmente o da ION Niterói!
> 
> Se tiver qualquer dúvida sobre os editais, precisar de ajuda para entender algum ponto ou quiser buscar outras opções, é só me dizer! Estou aqui para te ajudar em cada passo dessa jornada.
> 
> Vamos juntos nessa! 💪 Boa sorte!


--------------------------------------------------------------
